### 0 导包并使用GPU加速训练

In [ ]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch.optim as optim
import matplotlib.pyplot as plt

# 检查是否有可用的 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

### 1 数据预处理

In [ ]:
class MyDataset:
    def __init__(self, train_path, test_path, target_column='cnt'):
        # 加载并处理数据
        self.datasets = {
            'train': self.process_data(train_path, target_column),
            'test': self.process_data(test_path, target_column)
        }
    
    def load_data(self, file_path):
        # 加载数据
        return pd.read_csv(file_path)
    
    def preprocess_data(self, data):
        # 提取时间特征
        data['hour'] = data['hr']
        data['weather'] = data['weathersit']
        data['Actual_temperature'] = data['temp']
        data['Feeling_temperature'] = data['atemp']
        data['humidity'] = data['hum']
        # add 'yr','mnth','holiday','workingday','windspeed' 
        # 删除相关度低的列
        data = data.drop(columns=['instant', 'dteday', 'hr', 'weathersit', 'temp', 'atemp', 'hum'])
        # 删除缺失值
        data = data.dropna()
        return data
    
    def process_data(self, file_path, target_column):
        # 加载数据
        data = self.load_data(file_path)
        # 预处理数据
        data = self.preprocess_data(data)
        # 提取目标变量
        self.target_column = target_column
        y = np.log(data[self.target_column].values)
        # 删除目标变量
        X = data.drop(columns=[self.target_column])
        
        return {
            'X': X.values,
            'y': y
        }
# ShortDataset 类：用于短期预测（96小时）数据封装
class ShortDataset(Dataset):
    def __init__(self, X, y, seq_len=96, future_len=96):
        """
        构造方法，初始化数据和序列长度。
        
        :param X: 特征数据 (ndarray)，形状为 (num_samples, num_features)
        :param y: 目标数据 (ndarray)，形状为 (num_samples,)
        :param seq_len: 输入序列长度 (int)，默认过去96小时
        :param future_len: 预测未来的时间长度，默认未来96小时
        """
        self.X = X
        self.y = y
        self.seq_len = seq_len  # 过去的数据窗口长度
        self.future_len = future_len  # 预测未来的数据窗口长度
    
    def __len__(self):
        """
        返回数据集的长度，减去seq_len，确保我们可以生成完整的时间序列对。
        """
        return len(self.X) - self.seq_len - self.future_len
    
    def __getitem__(self, idx):
        """
        获取一个样本数据： 
        - X_seq: 输入特征数据序列（过去96小时） 
        - y_seq: 目标值（未来96小时）
        
        :param idx: 数据索引
        :return: 输入序列和对应的目标值
        """
        # 获取过去seq_len小时的数据
        X_seq = self.X[idx:idx + self.seq_len]
        
        # 获取未来future_len小时的目标数据（预测）
        y_seq = self.y[idx + self.seq_len:idx + self.seq_len + self.future_len]
        
        return torch.tensor(X_seq, dtype=torch.float32).to(device), torch.tensor(y_seq, dtype=torch.float32).to(device)

# LongDataset 类：用于长期预测（240小时）数据封装
class LongDataset(Dataset):
    def __init__(self, X, y, seq_len=96, future_len=240):
        """
        构造方法，初始化数据和序列长度。
        
        :param X: 特征数据 (ndarray)，形状为 (num_samples, num_features)
        :param y: 目标数据 (ndarray)，形状为 (num_samples,)
        :param seq_len: 输入序列长度 (int)，默认过去96小时
        :param future_len: 预测未来的时间长度，默认未来240小时
        """
        self.X = X
        self.y = y
        self.seq_len = seq_len  # 过去的数据窗口长度
        self.future_len = future_len  # 预测未来的数据窗口长度
    
    def __len__(self):
        """
        返回数据集的长度，减去seq_len，确保我们可以生成完整的时间序列对。
        """
        return len(self.X) - self.seq_len - self.future_len
    
    def __getitem__(self, idx):
        """
        获取一个样本数据： 
        - X_seq: 输入特征数据序列（过去96小时） 
        - y_seq: 目标值（未来240小时）
        
        :param idx: 数据索引
        :return: 输入序列和对应的目标值
        """
        # 获取过去seq_len小时的数据
        X_seq = self.X[idx:idx + self.seq_len]
        
        # 获取未来future_len小时的目标数据（预测）
        y_seq = self.y[idx + self.seq_len:idx + self.seq_len + self.future_len]
        
        return torch.tensor(X_seq, dtype=torch.float32).to(device), torch.tensor(y_seq, dtype=torch.float32).to(device)
    
def create_shortDataloader(X, y, seq_len=96, future_len=96, batch_size=32):
    """
    创建训练集和验证集的 DataLoader。
    
    :param X: 特征数据 (ndarray)，形状为 (num_samples, num_features)
    :param y: 目标数据 (ndarray)，形状为 (num_samples,)
    :param seq_len: 输入序列长度 (int)，默认过去96小时
    :param future_len: 预测未来的时间长度，默认未来96小时
    :param batch_size: 批次大小 (int)
    :return: DataLoader 对象
    """
    dataset = ShortDataset(X, y, seq_len, future_len)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return dataloader

def create_longDataloader(X, y, seq_len=96, future_len=240, batch_size=16):
    """
    创建训练集和验证集的 DataLoader。
    
    :param X: 特征数据 (ndarray)，形状为 (num_samples, num_features)
    :param y: 目标数据 (ndarray)，形状为 (num_samples,)
    :param seq_len: 输入序列长度 (int)，默认过去96小时
    :param future_len: 预测未来的时间长度，默认未来240小时
    :param batch_size: 批次大小 (int)
    :return: DataLoader 对象
    """
    dataset = ShortDataset(X, y, seq_len, future_len)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return dataloader

# 使用方法
train_path = 'train_data.csv'
test_path = 'test_data.csv'

# 创建 MyDataset 实例
dataset = MyDataset(train_path, test_path)

X_train, y_train = dataset.datasets['train']['X'], dataset.datasets['train']['y']
X_test, y_test = dataset.datasets['test']['X'], dataset.datasets['test']['y']

# 创建短期预测训练集和测试集的DataLoader
short_train_loader = create_shortDataloader(X_train, y_train, seq_len=96, future_len=96, batch_size=32)
short_test_loader = create_shortDataloader(X_test, y_test, seq_len=96, future_len=96, batch_size=32)
# 创建长期预测训练集和测试集的DataLoader
long_train_loader = create_shortDataloader(X_train, y_train, seq_len=96, future_len=240, batch_size=16)
long_test_loader = create_shortDataloader(X_test, y_test, seq_len=96, future_len=240, batch_size=16)

### 2 不同模型的实现

#### 2.1 LSTM

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, num_features, input_size, output_size, hidden_size=128, num_layers=1):
        super(LSTMModel, self).__init__()
        self.num_features = num_features
        self.input_size = input_size  # 已知的时间点数
        self.output_size = output_size  # 预测的时间点数
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM层
        self.lstm = nn.LSTM(input_size=self.num_features, hidden_size=self.hidden_size, num_layers=self.num_layers, batch_first=True)
        
        # 全连接层，用于将LSTM的输出映射到预测值
        self.fc = nn.Linear(self.hidden_size, self.output_size)  # 输出多个时间步（output_size）

    def forward(self, x):
        # x的形状: (batch_size, seq_len, num_features)
        
        # LSTM前向传播
        lstm_out, (hn, cn) = self.lstm(x)  # lstm_out.shape = (batch_size, seq_len, hidden_size)
        
        # 对LSTM输出的每个时间步进行处理，预测多个时间步
        out = self.fc(lstm_out[:, -1, :]).squeeze(-1)  # 使用LSTM最后一个时间步的输出，进行预测

        # 输出最后的预测值（output_size 个时间步）
        return out


#### 2.2 Transformer

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, num_features, input_size, output_size, num_heads, num_layers, hidden_dim, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.input_size = input_size
        self.num_features = num_features
        self.output_size = output_size
        # Encoder Layer
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=hidden_dim, nhead=num_heads, dim_feedforward=hidden_dim, dropout=dropout
            ),
            num_layers=num_layers
        )
        # Input layer (embedding layer)
        self.input_layer = nn.Linear(num_features, hidden_dim)
        # Output layer
        self.output_layer = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        # 输入层 (embedding)
        x = self.input_layer(x)
        # Transformer编码器
        x = x.permute(1, 0, 2)  # (seq_len, batch_size, feature_dim)
        x = self.encoder(x)
        # 选择最后时刻的输出进行预测
        x = x[-1, :, :]
        # 输出层
        x = self.output_layer(x)
        return x

#### 2.3 N-BEATS

In [ ]:
class NBeatsBlock(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, block_type='trend'):
        super(NBeatsBlock, self).__init__()
        self.block_type = block_type
        self.hidden_size = hidden_size
        self.input_size = input_size  # input_size is now features * seq_len
        self.output_size = output_size
        
        # Fully connected layers
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        
        # For trend block, we add additional components if needed
        if self.block_type == 'trend':
            self.fc4 = nn.Linear(input_size, hidden_size)  # For trend-specific processing
        
    def forward(self, x):
        # Flatten the input (batch_size, seq_len, features) -> (batch_size, seq_len * features)
        batch_size, seq_len, features = x.size()
        x = x.view(batch_size, seq_len * features)  # Flatten to (batch_size, seq_len * features)
        
        # Block forward pass
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x


class NBeatsModel(nn.Module):
    def __init__(self, num_features, input_size, output_size, hidden_size, num_blocks=3, block_type='trend'):
        super(NBeatsModel, self).__init__()
        self.num_blocks = num_blocks
        self.blocks = nn.ModuleList([NBeatsBlock(input_size * num_features, output_size, hidden_size, block_type)
                                     for _ in range(num_blocks)])
        
    def forward(self, x):
        # Pass the input through each block and aggregate the predictions
        predictions = []
        for block in self.blocks:
            predictions.append(block(x))
        
        # Combine predictions from all blocks, can apply averaging or other fusion methods
        return torch.mean(torch.stack(predictions), dim=0)

### 3 模型训练与评估

In [ ]:
def train_model(model, train_loader, val_loader, epochs=5, lr=0.001, log_print=print):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            # 将数据移到 GPU
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            # 前向传播
            y_pred = model(X_batch)
            # 计算损失
            loss = criterion(y_pred, y_batch)
            train_loss += loss.item()
            # 反向传播
            loss.backward()
            optimizer.step()

        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                # 将数据移到 GPU
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                y_pred = model(X_batch)
                loss = criterion(y_pred, y_batch)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        log_print(f"Epoch [{epoch + 1}/{epochs}], Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

    return train_losses, val_losses

def evaluate_model(model, test_loader, log_print=print):
    model.eval()
    predictions = []
    true_values = []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            # 将数据移到 GPU
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            # print(X_batch.shape)
            y_pred = model(X_batch)
            predictions.append(y_pred.cpu().tolist())  # 将结果从 GPU 转回 CPU
            true_values.append(y_batch.cpu().tolist())  # 将真实值从 GPU 转回 CPU

    predictions = np.exp(np.concatenate(predictions, axis=0))
    true_values = np.exp(np.concatenate(true_values, axis=0))

    # 计算均方误差和平均绝对误差
    mse = mean_squared_error(true_values, predictions)
    mae = mean_absolute_error(true_values, predictions)

    log_print(f"Evaluation Results: MSE = {mse:.4f}, MAE = {mae:.4f}")
    return mse, mae, predictions, true_values

def run_experiments(model_type, train_loader, val_loader, test_loader, num_features=14, input_size=96, output_size=96, hidden_dim=128, num_heads=3, num_layers=3, num_blocks=3, block_type='seasonality', num_experiments=5, epochs=20, lr=0.001, log_file='log.txt'):
    mse_scores = []
    mae_scores = []
    best_model = None
    best_mse = float('inf')  # 用于跟踪最佳模型
    best_predictions = None
    best_true_values = None

    # 打开日志文件
    with open(log_file, "w") as log:
        def log_print(*args, **kwargs):
            """将输出写入日志文件和控制台"""
            print(*args, **kwargs)
            print(*args, **kwargs, file=log)

        for i in range(num_experiments):
            log_print(f"Experiment {i + 1}/{num_experiments}")

            # 初始化模型
            if model_type == 'LSTM':
                model = LSTMModel(num_features=num_features,input_size=input_size, hidden_size=hidden_dim, output_size=output_size, num_layers=num_layers).to(device)
            elif model_type == 'Transformer':
                model = TransformerModel(num_features=num_features, input_size=input_size, output_size=output_size, num_heads=num_heads, num_layers=num_layers, hidden_dim=hidden_dim).to(device)
            else:
                model = NBeatsModel(num_features=num_features, input_size=input_size, output_size=output_size, hidden_size=hidden_dim,num_blocks=num_blocks,block_type=block_type).to(device)

            # 训练模型
            log_print("Training model...")
            train_losses, val_losses = train_model(model, train_loader, val_loader, epochs=epochs, lr=lr, log_print=log_print)

            # 评估模型
            log_print("Evaluating model...")
            mse, mae, predictions, true_values = evaluate_model(model, test_loader, log_print=log_print)

            # 保存结果
            mse_scores.append(mse)
            mae_scores.append(mae)

            # 更新最佳模型
            if mse < best_mse:
                best_mse = mse
                best_model = model
                best_predictions = predictions
                best_true_values = true_values

            log_print(f"Experiment {i + 1} Results: MSE = {mse:.4f}, MAE = {mae:.4f}\n")
        
        # 计算平均值和标准差
        mse_mean = np.mean(mse_scores)
        mse_std = np.std(mse_scores)
        mae_mean = np.mean(mae_scores)
        mae_std = np.std(mae_scores)

        log_print("Overall Results:")
        log_print(f"MSE: Mean = {mse_mean:.4f}, Std = {mse_std:.4f}")
        log_print(f"MAE: Mean = {mae_mean:.4f}, Std = {mae_std:.4f}")
        log_print("\nLogs saved to " + log_file + '.')

    return best_model, mse_mean, mse_std, mae_mean, mae_std, best_predictions, best_true_values


### 4 绘制结果对比图

In [ ]:
def plot_predictions(predictions, true_values, num_samples_to_plot=1,mode=96):
    """
    绘制预测值和真实值的对比图
    
    :param predictions: 预测结果，形状为 (num_samples, future_len)
    :param true_values: 真实值，形状为 (num_samples, future_len)
    :param num_samples_to_plot: 选择的样本数量，默认绘制第一个样本
    """
    plt.figure(figsize=(10, 6))
    
    # 只绘制前 num_samples_to_plot 个样本的预测和真实值对比
    for i in range(num_samples_to_plot):
        plt.plot(true_values[i], label=f"Ground Truth {i+1}", linestyle='-', color='b')
        plt.plot(predictions[i], label=f"Prediction {i+1}", linestyle='--', color='r')
    
    plt.xlabel("Time ("+ str(mode) +" hours)")
    plt.ylabel("Total Rental Count (cnt)")
    plt.legend()
    plt.title("Bike Rental Prediction vs Ground Truth")
    plt.show()

def plot_all_predictions(predictions, true_values, interval=96):
    """
    从每个样本中每隔指定时间步（如 96 个）选取一个值后合并，绘制预测值和真实值。
    
    :param predictions: 预测结果，形状为 (num_samples, seq_len)
    :param true_values: 真实值，形状为 (num_samples, seq_len)
    :param interval: 从每个样本中采样的间隔，默认每隔 96 个时间步采样一个值
    """
    # 从每个样本中采样
    sampled_predictions = predictions[:, ::interval].flatten()
    sampled_true_values = true_values[:, ::interval].flatten()
    
    print(f"Number of sampled points (per sample): {predictions.shape[1] // interval}")
    print(f"Total sampled points: {len(sampled_predictions)}")
    
    # 绘图
    plt.figure(figsize=(12, 6))
    plt.plot(sampled_true_values, label="Sampled Ground Truth", linestyle='-', color='b', alpha=0.7)
    plt.plot(sampled_predictions, label="Sampled Prediction", linestyle='--', color='r', alpha=0.7)
    
    plt.xlabel(f"Sampled Time Steps (Interval: {interval})")
    plt.ylabel("Total Rental Count (cnt)")
    plt.legend()
    plt.title("Bike Rental Prediction vs Ground Truth (Sampled)")
    plt.grid(True)
    plt.show()

### 5 短期预测

#### 5.1 LSTM

In [ ]:

# 执行多次实验，获取最佳模型及其性能指标
best_model, mse_mean, mse_std, mae_mean, mae_std, best_predictions, best_true_values = run_experiments(
    model_type='LSTM',
    train_loader=short_train_loader,
    val_loader=short_test_loader,  # 验证集
    test_loader=short_test_loader,  # 测试集
    num_features=X_train.shape[1],  # 特征数量
    input_size=96,
    output_size=96,
    num_layers=3,
    hidden_dim=128,
    num_experiments=5,
    epochs=40,
    lr=0.001,
    log_file="LSTM_short_prediction_process.txt"
)

# 绘制最佳模型的预测结果与真实值的对比图
plot_predictions(best_predictions, best_true_values, num_samples_to_plot=1, mode=96)
plot_all_predictions(best_predictions, best_true_values, interval=96)

#### 5.2 Transformer

In [ ]:
# 执行多次实验，获取最佳模型及其性能指标
best_model, mse_mean, mse_std, mae_mean, mae_std, best_predictions, best_true_values = run_experiments(
    model_type='Transformer',
    train_loader=short_train_loader,
    val_loader=short_test_loader,  # 验证集
    test_loader=short_test_loader,  # 测试集
    num_features=X_train.shape[1],  # 特征数量
    input_size=96,
    output_size=96,
    num_heads=8,
    num_layers=3,
    hidden_dim=128,
    num_experiments=5,
    epochs=40,
    lr=0.001,
    log_file="transformer_short_prediction_process.txt"
)

# 绘制最佳模型的预测结果与真实值的对比图
plot_predictions(best_predictions, best_true_values, num_samples_to_plot=1, mode=96)
plot_all_predictions(best_predictions, best_true_values, interval=96)


### 5.3 N-BEATS

In [ ]:
# 执行多次实验，获取最佳模型及其性能指标
best_model, mse_mean, mse_std, mae_mean, mae_std, best_predictions, best_true_values = run_experiments(
    model_type='N-BEATS',
    train_loader=short_train_loader,
    val_loader=short_test_loader,  # 验证集
    test_loader=short_test_loader,  # 测试集
    num_features=X_train.shape[1],  # 特征数量
    input_size=96,
    output_size=96,
    hidden_dim=256,
    num_blocks=6,
    block_type='seasonality',
    num_experiments=5,
    epochs=40,
    lr=0.001,
    log_file="N-BEATS_short_prediction_process.txt"
)


# 绘制最佳模型的预测结果与真实值的对比图
plot_predictions(best_predictions, best_true_values, num_samples_to_plot=1, mode=96)
plot_all_predictions(best_predictions, best_true_values, interval=96)


### 6 长期预测

#### 6.1 LSTM

In [ ]:
# 执行多次实验，获取最佳模型及其性能指标
best_model, mse_mean, mse_std, mae_mean, mae_std, best_predictions, best_true_values = run_experiments(
    model_type='LSTM',
    train_loader=long_train_loader,
    val_loader=long_test_loader,  # 验证集
    test_loader=long_test_loader,  # 测试集
    num_features=X_train.shape[1],  # 特征数量
    input_size=96,
    output_size=240,
    num_layers=3,
    hidden_dim=256,
    num_experiments=5,
    epochs=80,
    lr=0.001,
    log_file="LSTM_long_prediction_process.txt"
)

# 绘制最佳模型的预测结果与真实值的对比图
plot_predictions(best_predictions, best_true_values, num_samples_to_plot=1, mode=96)
plot_all_predictions(best_predictions, best_true_values, interval=240)

#### 6.2 Transformer

In [ ]:
# 执行多次实验，获取最佳模型及其性能指标
best_model, mse_mean, mse_std, mae_mean, mae_std, best_predictions, best_true_values = run_experiments(
    model_type='Transformer',
    train_loader=long_train_loader,
    val_loader=long_test_loader,  # 验证集
    test_loader=long_test_loader,  # 测试集
    num_features=X_train.shape[1],  # 特征数量
    input_size=96,
    output_size=240,
    num_heads=4,
    num_layers=3,
    hidden_dim=128,
    num_experiments=5,
    epochs=80,
    lr=0.001,
    log_file="transformer_long_prediction_process.txt"
)


# 绘制最佳模型的预测结果与真实值的对比图
plot_predictions(best_predictions, best_true_values, num_samples_to_plot=1, mode=240)
plot_all_predictions(best_predictions, best_true_values, interval=240)

#### 6.3 N-BEATS

In [ ]:
# 执行多次实验，获取最佳模型及其性能指标
best_model, mse_mean, mse_std, mae_mean, mae_std, best_predictions, best_true_values = run_experiments(
    model_type='N-BEATS',
    train_loader=long_train_loader,
    val_loader=long_test_loader,  # 验证集
    test_loader=long_test_loader,  # 测试集
    num_features=X_train.shape[1],  # 特征数量
    input_size=96,
    output_size=240,
    hidden_dim=128,
    num_blocks=3,
    block_type='seasonality',
    num_experiments=5,
    epochs=80,
    lr=0.001,
    log_file="N-BEATS_long_prediction_process.txt"
)


# 绘制最佳模型的预测结果与真实值的对比图
plot_predictions(best_predictions, best_true_values, num_samples_to_plot=1, mode=240)
plot_all_predictions(best_predictions, best_true_values, interval=240)
